In [ ]:
# Import scipy optimize
!pip install scipy
from scipy.optimize import minimize

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Define Flexural Stress

In [ ]:
def flex_stress(param, M): # NOTE: Append y_bar and I to param?
  A = np.array([param[0]*param[1], param[2]*param[3], param[2]*param[3], param[4]*param[5], param[4]*param[5], param[6]*param[7]])
  centroids = np.array([param[5]+param[7]+param[1]/2, param[5]+param[7]-param[3]/2, param[5]+param[7]-param[3]/2, param[7] + param[5]/2, param[7] + param[5]/2, param[7]/2])
  y_bar = np.sum(A * centroids) / np.sum(A)
  I = (param[0] * param[1]**3)/12 + 2 * (param[2] * param[3]**3)/12 + 2*(param[4] * param[5]**3)/12 + (param[6] * param[7]**3)/12 + np.sum(A * (y_bar - centroids)**2)

  return M*(param[7] * param[5] * param[1] - y_bar) / I

In [ ]:
def minimize_flex_stress(param, M):
  return flex_stress(param, M)

#### Define Known Parameters

NOTE: Dimensions are 32” × 40” × 0.05” (813 mm × 1016 mm × 1.27 mm)
- For better results, set correct bounds to the parameters.

In [ ]:
# Define bounds of the parameters (1016)
num_shapes = 6
bounds = [(1.27, None), (1.27, 1.27), (1.27, 5),
          (1.27, 5), (1.27, 5), (1.27, None),
          (1.27, None), (1.27, 1.27)]

# Initial guess [[b], [h]] -- using the cross-section in Design 0
param = np.array([100, 1.27, 5, 1.27, 1.27, 73.73, 80, 1.27])

A = np.array([param[0]*param[1], param[2]*param[3], param[2]*param[3], param[4]*param[5], param[4]*param[5], param[6]*param[7]])
centroids = np.array([param[5]+param[7]+param[1]/2, param[5]+param[7]-param[3]/2, param[5]+param[7]-param[3]/2, param[7] + param[5]/2, param[7] + param[5]/2, param[7]/2])
y_bar = np.sum(A * centroids) / np.sum(A)
I = (param[0] * param[1]**3)/12 + 2 * (param[2] * param[3]**3)/12 + 2*(param[4] * param[5]**3)/12 + (param[6] * param[7]**3)/12 + np.sum(A * (y_bar - centroids)**2)

print(f'Initial y_bar: {y_bar}')
print(f'Initial I: {I}')

# Constraints dict
def area_constraint(A: np.ndarray) -> float:
  A_of_board = 1016 - np.sum(A)

def base_constraint(param, i):
    # Ensure width of shape i is within bounds, for example, 1.27 <= b_i <= 813
    return param[i] - 1.27

def height_constraint(param, i):
    # Ensure height of shape i is within bounds, for example, 1.27 <= h_i <= 1016
    return param[i] - 1.27

def constraints(param):
  constraints = []
  for i in range(len(param)):
    if i % 2 == 0: # Base
      constraints.append({'type': 'ineq', 'fun': lambda param: base_constraint(param, i)})
      constraints.append({'type': 'ineq', 'fun': lambda param, i=i: 813 - param[i]})  # Ensure param[i] <= 813
    else: # Height
      constraints.append({'type': 'ineq', 'fun': lambda param: height_constraint(param, i)})
      constraints.append({'type': 'ineq', 'fun': lambda param, i=i: 1016 - param[i]})  # Ensure param[i] <= 1016

  return constraints

cons = constraints(param)

# Given max bending moment -- calculated in Design 0
M = 76.6e3

# Minimize
min = minimize(minimize_flex_stress, param, args=(M,), bounds=bounds, constraints=cons)

# Optimal parameters
print(f'Optimal: {min.x}')

Initial y_bar: 41.43109435192319
Initial I: 418352.2089994236
Optimal: [ 240.68413539    1.27          5.            5.            5.
 1016.          190.59137761    1.27      ]


In [ ]:
import numpy as np

arr = np.array([ [1, 0], [0, 1], [0, -1] ])
print(arr.T)

[[ 1  0  0]
 [ 0  1 -1]]
